# Cómo construir un chatbot simple con memoria almacenada usando LangChain
* Aplicación de Chatbot simple con LLM.
    * Podrá mantener una conversación.
    * Recordará interacciones previas: tendrá memoria.
    * Podrá almacenar la memoria en un archivo JSON.

Una aplicación de chatbot es un software diseñado para simular conversaciones con usuarios humanos. Utiliza inteligencia artificial (IA) para entender lo que alguien está diciendo y responder con respuestas apropiadas. Estas aplicaciones pueden usarse para diversos propósitos como soporte al cliente, proporcionar información o entretenimiento. Esencialmente, es como enviar mensajes de texto con un programa que puede responder preguntas y ayudar con tareas.

## Conceptos incluidos
* Modelo de Chat vs. Modelo LLM:
    * El modelo de chat se basa en mensajes.
    * El modelo LLM se basa en texto sin formato.
* Historial de chat: permite que el modelo de chat recuerde interacciones previas.

## Configuración

#### Después de descargar el código del repositorio de GitHub en tu computadora
En la terminal:
* cd nombre_del_proyecto
* pyenv local 3.11.4
* poetry install
* poetry shell

#### Para abrir el notebook con Jupyter Notebooks
En la terminal:
* jupyter lab

Ve a la carpeta de notebooks y abre el notebook correcto.

#### Para ver el código en Visual Studio Code o tu editor preferido.
* Abre Visual Studio Code o tu editor preferido.
* Abre la carpeta del proyecto.
* Abre el archivo 001-simple-chatbot.py.

## Crea tu archivo .env
* En el repositorio de GitHub hemos incluido un archivo llamado .env.example.
* Renombra ese archivo a .env y aquí es donde agregarás tus claves API confidenciales. Recuerda incluir:
* OPENAI_API_KEY=tu_clave_api_de_openai
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=tu_clave_api_de_langchain
* LANGCHAIN_PROJECT=tu_nombre_de_proyecto

Llamaremos a nuestro proyecto LangSmith **001-simple-chatbot**.

## Truco para evitar las molestas advertencias de depreciación de LangChain

En este ejercicio usaremos la cadena heredada de LangChain, LLMChain. Funciona bien, pero LangChain muestra una molesta advertencia de depreciación. Para evitarla, ingresaremos el siguiente código:

In [ ]:
import warnings
from langchain._api import LangChainDeprecationWarning

# Ignorar las advertencias de depreciación específicas de LangChain
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

## Conéctate con el archivo .env ubicado en el mismo directorio de este notebook

Si estás usando el entorno pre-cargado de poetry shell, no necesitas instalar el siguiente paquete porque ya está pre-cargado:

In [ ]:
#!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

# Cargar las variables de entorno desde el archivo .env
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Instalar LangChain

Si estás usando el entorno pre-cargado de poetry shell, no necesitas instalar el siguiente paquete porque ya está pre-cargado:

In [ ]:
#!pip install langchain

## Conéctate con un LLM y comienza una conversación

Si estás usando el entorno pre-cargado de poetry shell, no necesitas instalar el siguiente paquete porque ya está pre-cargado:

In [ ]:

#!pip install langchain-openai

* Para este proyecto, usaremos gpt-3.5-turbo de OpenAI.

In [ ]:

from langchain_openai import ChatOpenAI

# Crear una instancia del chatbot usando el modelo gpt-3.5-turbo
chatbot = ChatOpenAI(model="gpt-3.5-turbo")

In [8]:
from langchain_ollama import ChatOllama

chatbot = ChatOllama(model='llama3:latest')

* Mensaje humano: la entrada del usuario.

In [7]:

from langchain_core.messages import HumanMessage

# Crear un mensaje para el chatbot
messagesToTheChatbot = [
    HumanMessage(content="Mi color favorito es azul."),
]

#### Llamar al modelo de chat (el LLM)

In [9]:

# Invocar el chatbot con los mensajes proporcionados
chatbot.invoke(messagesToTheChatbot)

AIMessage(content='¡genial! El azul es un color muy hermoso y popular. ¿Te gusta porque es un color que evoca paz, tranquilidad o algo más específico?', response_metadata={'model': 'llama3:latest', 'created_at': '2025-08-13T17:28:52.458308988Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3016507942, 'load_duration': 2132460901, 'prompt_eval_count': 18, 'prompt_eval_duration': 122273956, 'eval_count': 38, 'eval_duration': 761334681, 'message': Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=None)}, id='run-f9440246-8d01-447a-9dae-6935d9d85727-0', usage_metadata={'input_tokens': 18, 'output_tokens': 38, 'total_tokens': 56})

#### Seguimiento de la operación en LangSmith
* [Abre LangSmith aquí](https://smith.langchain.com/)

## Verifica si el chatbot recuerda tu color favorito.

In [10]:

# Preguntar al chatbot sobre el color favorito del usuario
chatbot.invoke([
    HumanMessage(content="¿Cuál es mi color favorito?")
])

AIMessage(content='¡haha! ¡Lo siento, pero como soy una inteligencia artificial, no tengo acceso a información personal sobre ti y no puedo saber cuál es tu color favorito! ¿Quieres decirme cuál es el tuyo?', response_metadata={'model': 'llama3:latest', 'created_at': '2025-08-13T17:28:54.193839709Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1105358235, 'load_duration': 55374846, 'prompt_eval_count': 19, 'prompt_eval_duration': 28725467, 'eval_count': 50, 'eval_duration': 1020810282, 'message': Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=None)}, id='run-7a853e16-3a39-477d-8647-346e93a15d2e-0', usage_metadata={'input_tokens': 19, 'output_tokens': 50, 'total_tokens': 69})

* Como puedes ver, nuestro chatbot no puede recordar nuestra interacción previa.

## Agreguemos memoria a nuestro chatbot

Si estás usando el entorno pre-cargado de poetry shell, no necesitas instalar el siguiente paquete porque ya está pre-cargado:

In [ ]:

#!pip install langchain_community

In [2]:
# Importar las bibliotecas necesarias para agregar memoria al chatbot
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory



In [3]:
# Configurar memoria para el chatbot usando un archivo JSON
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)

In [ ]:
# Crear un prompt para el chatbot con soporte de memoria
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [ ]:
# Crear una cadena LLM con memoria y el prompt configurado
chain = LLMChain(
    llm=chatbot,
    prompt=prompt,
    memory=memory
)

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv053024-p2/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
# Probar la cadena con un mensaje inicial
chain.invoke("¡Hola!")

{'content': 'hello!',
 'messages': [],
 'text': 'Hello! How can I assist you today?'}

In [ ]:
# Probar la cadena con un mensaje que incluye información personal
chain.invoke("Mi nombre es Julio")

{'content': 'my name is Julio',
 'messages': [HumanMessage(content='hello!'),
  AIMessage(content='Hello! How can I assist you today?')],
 'text': 'Nice to meet you, Julio! How can I help you today?'}

In [ ]:
# Preguntar al chatbot sobre la información almacenada en la memoria
chain.invoke("¿Cuál es mi nombre?")

{'content': 'what is my name?',
 'messages': [HumanMessage(content='hello!'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='my name is Julio'),
  AIMessage(content='Nice to meet you, Julio! How can I help you today?')],
 'text': 'Your name is Julio.'}

* Revisa el archivo messages.json en el directorio raíz.
* Esto es solo un ejemplo simple, en el mundo real probablemente no guardarás tu memoria en un archivo JSON.
* Y recuerda: la ventana de contexto es limitada y afecta el costo de usar la API de chatGPT.

## Cómo ejecutar el código desde Visual Studio Code
* En Visual Studio Code, ve al archivo 001-simple-chatbot.py.
* En la terminal, asegúrate de estar en el directorio del archivo y ejecuta:
    * python 001-simple-chatbot.py